In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("transfusion_5%.csv")

In [3]:
df

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,12500.0,98.0,1.0
1,NaN,13.0,3250.0,NaN,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,NaN,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,NaN,2.0,500.0,38.0,0.0
744,21.0,2.0,500.0,52.0,0.0
745,23.0,NaN,750.0,62.0,0.0
746,39.0,1.0,250.0,39.0,0.0


In [4]:
df.dtypes

Recency (months)                              float64
Frequency (times)                             float64
Monetary (c.c. blood)                         float64
Time (months)                                 float64
whether he/she donated blood in March 2007    float64
dtype: object

In [5]:
df.isnull().sum()

Recency (months)                              46
Frequency (times)                             46
Monetary (c.c. blood)                         55
Time (months)                                 40
whether he/she donated blood in March 2007     0
dtype: int64

In [6]:
df.isnull().sum().sum()

187

In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Recency (months)                              6.149733
Frequency (times)                             6.149733
Monetary (c.c. blood)                         7.352941
Time (months)                                 5.347594
whether he/she donated blood in March 2007    0.000000
dtype: float64

In [8]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 3740
nilai hilang : 187
persentase : 5.0


In [9]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [10]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,7)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
    
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [11]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [12]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.0,50.0,12500.0,98.0,1.0
1,0.0,13.0,3250.0,0.0,1.0
2,1.0,16.0,4000.0,35.0,1.0
3,2.0,0.0,5000.0,45.0,1.0
4,1.0,24.0,6000.0,77.0,0.0
...,...,...,...,...,...
743,0.0,2.0,500.0,38.0,0.0
744,21.0,2.0,500.0,52.0,0.0
745,23.0,0.0,750.0,62.0,0.0
746,39.0,1.0,250.0,39.0,0.0


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)
dummy

Kolom : 1
[1.0, 2.0, 2.0, 4.0, 12.0, 14.0, 21.0]
min :  1.0
max :  21.0
mean :  8.0
fuzzy :  [(12.0, 0.6923076923076923), (2.0, 0.14285714285714285), (4.0, 0.42857142857142855), (1.0, 0.0), (2.0, 0.14285714285714285), (14.0, 0.5384615384615384), (21.0, 0.0)]
fuzzy :  None
pembilang :  18.13186813186813
penyebut :  1.9450549450549448
rata-rata tertimbang :  9.322033898305087
 
[2.0, 2.0, 2.0, 2.0, 4.0, 14.0, 14.0]
min :  2.0
max :  14.0
mean :  5.714285714285714
fuzzy :  [(2.0, 0.0), (2.0, 0.0), (2.0, 0.0), (2.0, 0.0), (4.0, 0.5384615384615384), (14.0, 0.0), (14.0, 0.0)]
fuzzy :  None
pembilang :  2.1538461538461537
penyebut :  0.5384615384615384
rata-rata tertimbang :  4.0
 
[1.0, 2.0, 2.0, 7.0, 8.0, 12.0, 23.0]
min :  1.0
max :  23.0
mean :  7.857142857142857
fuzzy :  [(23.0, 0.0), (2.0, 0.14583333333333334), (8.0, 0.9905660377358491), (12.0, 0.7264150943396227), (7.0, 0.875), (2.0, 0.14583333333333334), (1.0, 0.0)]
fuzzy :  None
pembilang :  23.349842767295602
penyebut :  2.883647798

[0.0, 2.0, 14.0, 14.0, 16.0, 16.0, 21.0]
min :  0.0
max :  21.0
mean :  11.857142857142858
fuzzy :  [(0.0, 0.0), (14.0, 0.765625), (16.0, 0.546875), (14.0, 0.765625), (2.0, 0.1686746987951807), (16.0, 0.546875), (21.0, 0.0)]
fuzzy :  None
pembilang :  39.27484939759036
penyebut :  2.7936746987951806
rata-rata tertimbang :  14.058490566037735
 
[0.0, 2.0, 4.0, 11.0, 14.0, 14.0, 16.0]
min :  0.0
max :  16.0
mean :  8.714285714285714
fuzzy :  [(0.0, 0.0), (2.0, 0.2295081967213115), (16.0, 0.0), (11.0, 0.6862745098039215), (14.0, 0.2745098039215686), (4.0, 0.459016393442623), (14.0, 0.2745098039215686)]
fuzzy :  None
pembilang :  17.530376084860173
penyebut :  1.9238187078109932
rata-rata tertimbang :  9.112280701754386
 
[0.0, 2.0, 2.0, 3.0, 11.0, 21.0, 21.0]
min :  0.0
max :  21.0
mean :  8.571428571428571
fuzzy :  [(0.0, 0.0), (11.0, 0.8045977011494253), (21.0, 0.0), (2.0, 0.23333333333333334), (21.0, 0.0), (2.0, 0.23333333333333334), (3.0, 0.35000000000000003)]
fuzzy :  None
pembilang 

[18.0, 19.0, 19.0, 20.0, 21.0, 21.0, 22.0]
min :  18.0
max :  22.0
mean :  20.0
fuzzy :  [(20.0, 1.0), (21.0, 0.5), (21.0, 0.5), (19.0, 0.5), (19.0, 0.5), (22.0, 0.0), (18.0, 0.0)]
fuzzy :  None
pembilang :  60.0
penyebut :  3.0
rata-rata tertimbang :  20.0
 
[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
median :  4
min :  6.0
max :  6.0
mean :  6.0
fuzzy :  [(6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0)]
fuzzy :  6.0
pembilang :  252.0
penyebut :  42.0
rata-rata tertimbang :  6.0
 
[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0]
median :  4
min :  8.0
max :  8.0
mean :  8.0
fuzzy :  [(8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0), (8.0, 8.0)]
fuzzy :  8.0
pembilang :  448.0
penyebut :  56.0
rata-rata tertimbang :  8.0
 
[6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
median :  4
min :  6.0
max :  6.0
mean :  6.0
fuzzy :  [(6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0), (6.0, 6.0)]
fuzzy :  6.0
pembilang :  252.0
penyebut :  42.0
rata-

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
median :  4
min :  0.0
max :  7.0
mean :  6.0
fuzzy :  [(0.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0), (7.0, 7.0)]
fuzzy :  7.0
pembilang :  294.0
penyebut :  49.0
rata-rata tertimbang :  7.0
 
[0.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]
median :  4
min :  0.0
max :  3.0
mean :  2.5714285714285716
fuzzy :  [(0.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0), (3.0, 3.0)]
fuzzy :  3.0
pembilang :  54.0
penyebut :  21.0
rata-rata tertimbang :  3.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembi

[0.0, 0.0, 500.0, 500.0, 750.0, 1000.0, 1000.0]
min :  0.0
max :  1000.0
mean :  535.7142857142857
fuzzy :  [(0.0, 0.0), (750.0, 0.5384615384615384), (0.0, 0.0), (500.0, 0.9333333333333335), (1000.0, 0.0), (500.0, 0.9333333333333335), (1000.0, 0.0)]
fuzzy :  None
pembilang :  1337.1794871794873
penyebut :  2.4051282051282055
rata-rata tertimbang :  555.9701492537313
 
[1250.0, 1750.0, 2000.0, 2000.0, 2250.0, 2250.0, 2250.0]
min :  1250.0
max :  2250.0
mean :  1964.2857142857142
fuzzy :  [(2250.0, 0.0), (2000.0, 0.8749999999999998), (2250.0, 0.0), (2250.0, 0.0), (1750.0, 0.7000000000000001), (1250.0, 0.0), (2000.0, 0.8749999999999998)]
fuzzy :  None
pembilang :  4725.0
penyebut :  2.4499999999999993
rata-rata tertimbang :  1928.5714285714291
 
[1250.0, 1250.0, 1250.0, 1250.0, 1750.0, 1750.0, 1750.0]
median :  4
min :  1250.0
max :  1750.0
mean :  1464.2857142857142
fuzzy :  [(1750.0, 1250.0), (1250.0, 1250.0), (1250.0, 1250.0), (1250.0, 1250.0), (1250.0, 1250.0), (1750.0, 1250.0), (1750

...

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy : 

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy : 

[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.42857142857142855
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (1.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :

[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy : 

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.000000,50.0,12500.0,98.000000,1.0
1,9.322034,13.0,3250.0,44.859155,1.0
2,1.000000,16.0,4000.0,35.000000,1.0
3,2.000000,20.0,5000.0,45.000000,1.0
4,1.000000,24.0,6000.0,77.000000,1.0
...,...,...,...,...,...
743,11.865094,2.0,500.0,38.000000,1.0
744,21.000000,2.0,500.0,52.000000,1.0
745,23.000000,3.0,750.0,62.000000,1.0
746,39.000000,1.0,250.0,39.000000,1.0


In [15]:
dummy.describe()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
count,748.000000,748.000000,748.000000,748.000000,748.0
mean,9.580105,5.516043,1362.376044,33.780942,1.0
std,7.982576,5.838387,1445.919348,23.837718,0.0
min,1.000000,1.000000,250.000000,2.000000,1.0
25%,3.139535,2.000000,500.000000,16.000000,1.0
50%,8.048664,4.000000,1000.000000,28.000000,1.0
75%,14.000000,7.000000,1750.000000,48.000000,1.0
max,74.000000,50.000000,12500.000000,98.000000,1.0


# Klasifikasi

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [17]:
x = dummy.values
y = df.iloc[:, -1].values

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=72)

In [19]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [20]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[130  16]
 [ 28  13]]
0.7647058823529411


In [21]:
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[141   5]
 [ 35   6]]
0.786096256684492


In [22]:
classifier = MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.1, max_iter=100)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[142   4]
 [ 27  14]]
0.8342245989304813


C:\Users\User\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
